### Webscrapping with Pandas pd.read_html()  + ODBC on SQL Server with pyodbc

#### Source : Wikipedia / Liste des pays par Revenu (PIB par Habitant)

Cette liste des pays par PIB par habitant à parité du pouvoir d'achat comprend trois listes de pays du monde et de certains territoires classés selon la valeur de leur produit intérieur brut par habitant en parité de pouvoir d'achat (PPA) établies par les administrations suivantes : le Fonds monétaire international (FMI), la Banque mondiale (BM), et la Central Intelligence Agency (CIA).

Les écarts concernant le même pays peuvent être considérables selon l'organisme à l'origine de ces données. Ces variations s'expliquent par :

le fait que le PIB n'est pas toujours connu, il est souvent estimé par des spécialistes ;
le moment où ce PIB a été estimé (une projection antérieure à une crise économique change les chiffres) ;
l'estimation de la population locale, et qui est intégré dans cette population locale ;
les éléments choisis pour évaluer la parité de pouvoir d'achat recèlent une part de subjectivité.
Le classement ci-dessous n’est pas exact, notamment pour la Suisse dont le PIB par habitant est en réalité estimé à 86 850 USD pour l’année 2020 selon les statistiques du FMI1.

In [140]:
import numpy as np
import pandas as pd
# Beautifulsoup already installed into Anaconda environment

# Librairy PyODBC for SQL Server connection
import os
import pyodbc
# setup SQL Server connection session
server = '-PCSJN\DATAVIZ'  
database = 'Climbing_DW_BI'
connection_string= (
                    'DRIVER={SQL Server};SERVER='
                     + server
                     + ';DATABASE='
                     + database 
                     + ';Trusted_Connection=yes'                  
                    )

# Function for injecting dataframe content into SQL Server Table
def template_SQL_insert_values(table_name,dict):
    
    columns = "("
    for k in dict.keys():
        columns += str(k) + ','
    columns = columns[:-1] ### remove last comma
    columns += ")"
    
    req = f"""INSERT INTO {table_name} {columns} VALUES {tuple(dict.values())}"""
    return req

In [141]:
### 1er url
Wiki_Countries_PIB_Habitant = "https://fr.wikipedia.org/wiki/Liste_des_pays_par_PIB_(PPA)_par_habitant"
### récuperer toutes les tables html vu sur le site (url)
Dataset_Wiki_Countries_PIB_Habitant = pd.read_html(Wiki_Countries_PIB_Habitant)
type(Dataset_Wiki_Countries_PIB_Habitant), len(Dataset_Wiki_Countries_PIB_Habitant)

(list, 6)

In [142]:
Dataset_Wiki_Countries_PIB_Habitant[0]

,0,1,2
0,">$60,000 $50,000 – $60,000 $40,000 – $50,000 $...","$20,000 – $30,000 $10,000 – $20,000 $5,000 – $...","$1,000 – $2,500 <$1,000 Pas de données"


In [143]:
Dataset_Wiki_Countries_PIB_Habitant[1]

,Rang,Pays ou territoire,PIB par habitant[a]
0,01,Qatar,124 927
1,02,Luxembourg,109 192
2,03,Singapour,90 531
3,04,Brunei,76 743
4,05,Irlande,72 632
...,...,...,...
188,188,Niger,1 153
189,189,Liberia,867
190,190,République démocratique du Congo,808
191,191,Burundi,785


[1] Data from International Monetary Fund (IMF) 2017

In [144]:
Dataset_Wiki_Countries_PIB_Habitant[2]

,Rang,Pays ou territoire,PIB par habitant[a]
0,1,Luxembourg,142 214
1,2,Singapour,127 565
2,3,Irlande,126 905
3,4,Norvège,114 899
4,5,Qatar,114 648
...,...,...,...
179,180,Mozambique,1 468
180,181,Somalie,1 364
181,182,République démocratique du Congo,1 337
182,183,République centrafricaine,967


[2] Data from World Bank 2022

In [145]:
Dataset_Wiki_Countries_PIB_Habitant[3]

,Rang,Pays ou territoire,PIB par habitant[a]
0,1,Liechtenstein,139 100 (2009)
1,2,Qatar,124 900
2,3,Monaco,115 700 (2015)
3,4,Macao,114 400
4,5,Luxembourg,109 100
...,...,...,...
222,223,Tokelau,1 000 (1993)
223,224,Liberia,900
224,225,République démocratique du Congo,800
225,226,Burundi,800


[3] Data from CIA World Factbook 2017

In [146]:
Dataset_Wiki_Countries_PIB_Habitant[4]

,v · mListes de pays,v · mListes de pays.1
0,Démographie,Population 1700 1800 1907 Densité de populatio...
1,Économie,Monnaies Dette publique Dette extérieure Égali...
2,Éducation,Dépenses d'éducation Taux d'alphabétisation Cl...
3,Environnement,Performance environnementale Empreinte écologi...
4,Géographie,Continent Superficie forestière Sans cours d'e...
5,Militaire,Paix Dépenses militaires Forces militaires Ne ...
6,Politique,États membres de l'ONU Date d'acquisition de l...
7,Religion,Par religion Irréligion Bouddhisme Christianis...
8,Santé,Dépenses de santé Consommation de médicaments ...
9,Droit,Peine de mort exécutions les plus récentes Tau...


In [147]:
Dataset_Wiki_Countries_PIB_Habitant[5]

,v · mClassements des pays selon le produit intérieur brut (PIB),v · mClassements des pays selon le produit intérieur brut (PIB).1
0,Classements par PIB nominal,Par pays Par ville Par habitant
1,Classements par PIB à parité de pouvoir d'achat,Liste des pays par PIB (PPA) Par habitant Hist...
2,Indicateurs,PIB PNB PIB par habitant PIB réel PIB vert Val...
3,Liste des pays par volume d'exportation Indice...,Liste des pays par volume d'exportation Indice...


In [148]:
Dataset_Wiki_Countries_PIB_Habitant[2].columns

Index(['Rang', 'Pays ou territoire', 'PIB par habitant[a]'], dtype='object')

In [149]:
Dataset_Wiki_Countries_PIB_Habitant[2].columns = ['Rang', 'Nation', 'PIB_USD_par_habitant']

In [150]:
# retrouver via site web Invalid column name 'Côte d'Ivoire' pour script d'insertion de data ds SQL server
Dataset_Wiki_Countries_PIB_Habitant[2].loc[133,:]

Rang                              134
Nation                  Côte d'Ivoire
PIB_USD_par_habitant            6 538
Name: 133, dtype: object

In [151]:
# Remplacement 'Côte d'Ivoire'
Dataset_Wiki_Countries_PIB_Habitant[2].iloc[133,0] = 'Cote d Ivoire'

C:\Users\USER\AppData\Local\Temp\ipykernel_20000\2571056242.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Cote d Ivoire' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  Dataset_Wiki_Countries_PIB_Habitant[2].iloc[133,0] = 'Cote d Ivoire'


In [152]:
Dataset_Wiki_Countries_PIB_Habitant[2]

,Rang,Nation,PIB_USD_par_habitant
0,1,Luxembourg,142 214
1,2,Singapour,127 565
2,3,Irlande,126 905
3,4,Norvège,114 899
4,5,Qatar,114 648
...,...,...,...
179,180,Mozambique,1 468
180,181,Somalie,1 364
181,182,République démocratique du Congo,1 337
182,183,République centrafricaine,967


In [153]:
Dataset_Wiki_Countries_PIB_Habitant[2].dtypes

Rang                    object
Nation                  object
PIB_USD_par_habitant    object
dtype: object

In [154]:
Wiki_Demography_perNation_GDP_perInHabitant_WorldBank2022 = Dataset_Wiki_Countries_PIB_Habitant[2]

# script de création d'une table ds SQL 
create_table = """Create TABLE Wiki_Demography_perNation_GDP_perInHabitant_WorldBank2022
                                   (
                                        id INT IDENTITY PRIMARY KEY,
                                        Nation VARCHAR(100),
                                        Rang INT,                                        
                                        PIB_USD_par_habitant VARCHAR(50)                                                                                                                                     
                                   ) """

# launch session
cnxn = pyodbc.connect(connection_string,autocommit=True) # no cursor.commit as Autocommit already
cursor = cnxn.cursor() 

try: 
    # load script "create " 
    cursor.execute(create_table) 
    
    # population de la table ds SQL
    for i,row in Wiki_Demography_perNation_GDP_perInHabitant_WorldBank2022.iterrows(): 
        row_dict=row.to_dict()
        temp = template_SQL_insert_values('Wiki_Demography_perNation_GDP_perInHabitant_WorldBank2022',row_dict)
        cursor.execute(temp)
   
except Exception as err: 
    print('err:',err) 
try: 
    cursor.close()  # close session
except Exception as err: 
    print('err:',err)

err: ('42S22', "[42S22] [Microsoft][ODBC SQL Server Driver][SQL Server]Invalid column name 'Côte d'Ivoire'. (207) (SQLExecDirectW)")
